In [1]:
import os
import csv
import nltk
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

input_file = 'USvideos.csv'
# # plt.figure(figsize=(7.5, 4.25))
with open(input_file, 'r') as youtube_data:
    masterdata_csv = list(csv.reader(youtube_data))
    
masterdata_pd = pd.read_csv('USvideos.csv', index_col = None, na_values = ['NA'])

def run_query(query):
    return pd.read_sql_query(query,db)

def open_db(database):
    db = sqlite3.connect(database)
    db.execute('PRAGMA foreign_keys = ON;')
    return db
    

In [2]:
# youtube_pd = pd.read_csv('USvideos.csv', index_col = None, na_values = ['NA'])
masterdata_pd.head(n = 5)
#masterdata_pd.tail(n =5)
# filtered_yt = youtube_pd[(youtube_pd.title == 'something') & (youtube_pd.views > something else)]
#filtered_yt = youtube_pd.filter(items = ['title', 'views'])
# youtube_pd.describe() will summarize your data for you
# youtube_pd['likes'].describe()
# pandas has a merge function, like SQL joins. pd.merge(), supports left right inner outer
# .pivot allows for the creation or pivot tables by indicating a row x colum array (child poverty shown with city as the rows and year as the columns)

# Can I write derrived data back into a panda data set?
# Where does this fit in with the requirements to use SQL?
# How does this work library to library? Can I feed panda data into matplotlib? By subsetting, or indexing

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


Pandas - Panel Data

Rows are a ranges, columns are variables
pd.read_csv()
put the csv in the repo -- they want to see you download the data to the current working directory


In [3]:
try : 
    os.remove('youtube.db')
except FileNotFoundError:
    pass

db = open_db('youtube.db')
c = db.cursor()

rows = db.execute('pragma foreign_keys')
for row in rows:
    print(row)

create_statement_master = '''
    CREATE TABLE tblMasterData (
    id INTEGER AUTOIMCREMENT PRIMARY KEY,
    video_id TEXT,
    trending_date TEXT,
    title TEXT,
    channel_title TEXT,
    category_id INTEGER,
    publish_time INTEGER,
    tags TEXT,
    views INTEGER,
    likes INTEGER,
    dislikes INTEGER,
    comment_count INTEGER,
    thumbnail_link TEXT,
    comments_disabled TEXT,
    ratings_disabled TEXT,
    video_error_or_removed TEXT,
    description TEXT)
    '''

c.execute(create_statement_master)
#  ask how to avoid the cursor output
db.commit() # put in a finally (try, except, finally)

db.close() # put in a finally (try, except, finally)

# pd.read_sql_query("fun query stuff here") # ask about this

# run_query(create_statement)

(1,)


In [4]:
headers = masterdata_csv[0]
headers_dict = {}

for count, value in enumerate(headers, 1):
    headers_dict[value] = count -1
# to keep from 0 indexing my columns

del masterdata_csv[0]

In [5]:
db = open_db('youtube.db')
c = db.cursor()

c.executemany('''
INSERT INTO tblMasterData
(video_id,
trending_date,
title,
channel_title,
category_id,
publish_time,
tags,
views,
likes,
dislikes,
comment_count,
thumbnail_link,
comments_disabled,
ratings_disabled,
video_error_or_removed,
description)
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
''', 
masterdata_csv)

db.commit()

db.close()

In [6]:
db = open_db('youtube.db')
c = db.cursor()

create_statement_video = '''
    CREATE TABLE tblVideos (
    video_id TEXT PRIMARY KEY NOT NULL,
    title TEXT NOT NULL,
    channel_title TEXT NOT NULL,
    publish_time INTEGER NOT NULL,
    tags TEXT NOT NULL,
    thumbnail_link TEXT NOT NULL,
    comments_disabled TEXT NOT NULL,
    ratings_disabled TEXT NOT NULL,
    video_error_or_removed TEXT NOT NULL,
    description TEXT NOT NULL)
    '''

c.execute(create_statement_video)

create_statement_time = '''
    CREATE TABLE tblTime (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    video_id TEXT NOT NULL,
    trending_date TEXT NOT NULL,
    views INTEGER NOT NULL,
    likes INTEGER NOT NULL,
    dislikes INTEGER NOT NULL,
    comment_count INTEGER NOT NULL,
        FOREIGN KEY(video_id) REFERENCES tblVideos(video_id))
    '''

c.execute(create_statement_time)

db.commit()

db.close()

In [7]:
# Transform date column

In [8]:
for row in masterdata_csv:
    date = row[headers_dict['trending_date']]
    updated_date = "20" + date[:2]
    updated_date += "-"
    updated_date += date[6:]
    updated_date += "-"
    updated_date += date[3:5]
    row[headers_dict['trending_date']] = updated_date

In [9]:
video_data = []
time_data = []

for row in masterdata_csv:
    video_entry = [row[headers_dict['video_id']], 
                   row[headers_dict['title']], 
                   row[headers_dict['channel_title']], 
                   row[headers_dict['publish_time']],
                   row[headers_dict['tags']],
                   row[headers_dict['thumbnail_link']],
                   row[headers_dict['comments_disabled']],
                   row[headers_dict['ratings_disabled']],
                   row[headers_dict['video_error_or_removed']],
                   row[headers_dict['description']]
                  ]
    
    video_data.append(video_entry)
    
    time_entry = [row[headers_dict['video_id']], 
                  row[headers_dict['trending_date']], 
                  row[headers_dict['views']], 
                  row[headers_dict['likes']],
                  row[headers_dict['dislikes']],
                  row[headers_dict['comment_count']]
                 ]
    
    time_data.append(time_entry)

In [10]:
db = open_db('youtube.db')
c = db.cursor()

c.executemany('''
        INSERT OR REPLACE INTO tblVideos
        (video_id,
        title,
        channel_title,
        publish_time,
        tags,
        thumbnail_link,
        comments_disabled,
        ratings_disabled,
        video_error_or_removed,
        description)
        VALUES (?,?,?,?,?,?,?,?,?,?)
        ''', 
        video_data)

db.commit()
    
c.executemany('''
        INSERT INTO tblTime
        (video_id,
        trending_date,
        views,
        likes,
        dislikes,
        comment_count)
        VALUES (?,?,?,?,?,?)
        ''', 
        time_data)

db.commit()

db.close()

##Test for foreign key failure. 
It failed!!! :) 
...better have failed.

In [11]:
db = open_db('youtube.db')
c = db.cursor()

c.execute("INSERT INTO tblTime VALUES (?, ?, ?, ?, ?, ?, ?)", (None, "testy", "18.11.11", 1, 1, 1, 1))

db.commit()
db.close()


IntegrityError: FOREIGN KEY constraint failed

In [ ]:
class Video:
    
    def __init__(self, video_id, title):
        self.video_id = video_id
        self.title = title

    def title_text(self):
        
    def longest_word(self):

    def title_segements(self):


class Segment:
    
    def __init__(self, video_id, segment_id, segment):
        self.video_id = video_id
        self.segment_id = segment_id
        self.segment = segment
    
    def segment_structure(self):
    

        



To do:
- concat titles for fun analysis
- search nltk for stuff to make classes work
